In [ ]:
import os
from getpass import getpass
user = 'manigalati'
password = getpass(user)
os.environ['GITHUB_AUTH'] = user + ':' + password
!git clone https://$GITHUB_AUTH@github.com/PieraRiccio/Emotions-fromEEG-toArt

manigalati··········
Cloning into 'Emotions-fromEEG-toArt'...
remote: Enumerating objects: 199, done.
remote: Counting objects: 100% (199/199), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 199 (delta 54), reused 75 (delta 6), pack-reused 0
Receiving objects: 100% (199/199), 33.94 MiB | 33.29 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [ ]:
%cd Emotions-fromEEG-toArt/

/content/Emotions-fromEEG-toArt


## Install libs

In [ ]:
!pip install torch-scatter \
  torch-sparse \
  torch-cluster \
  torch-spline-conv \
  torch-geometric \
  -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
     |████████████████████████████████| 2.6MB 32.6MB/s 
     |████████████████████████████████| 1.5MB 31.7MB/s 
     |████████████████████████████████| 1.0MB 31.8MB/s 
     |████████████████████████████████| 389kB 36.3MB/s 
     |████████████████████████████████| 215kB 17.2MB/s 
     |████████████████████████████████| 235kB 37.0MB/s 
     |████████████████████████████████| 2.2MB 29.9MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.7.0-cp37-none-any.whl size=365386 sha256=e98346fdde8e09db5e29e92e410a4c6cb056ca41101ba64c8f34fe4dbd38a8c1
  Stored in directory: /root/.cache/pip/wheels/ce/fb/df/37cd43d1433e65b2d3417a71438404d0eeb4fbbfa40730931c
Successfully built torch-geometric


## WikiArt

In [ ]:
from wikiArt.wikiArt_utils import *

### Import

In [ ]:
!wget http://saifmohammad.com/WebDocs/WikiArt-Emotions.zip
!unzip WikiArt-Emotions.zip
!cp WikiArt-Emotions/WikiArt-Emotions-Ag4.tsv ./
download_wikiArtEmotion("wikiArt/dataset")
!rm -r WikiArt-Emotions
!rm -r __MACOSX/
!rm WikiArt-Emotions-Ag4.tsv
!rm WikiArt-Emotions.zip

--2021-04-11 11:36:47--  http://saifmohammad.com/WebDocs/WikiArt-Emotions.zip
Resolving saifmohammad.com (saifmohammad.com)... 192.185.17.122
Connecting to saifmohammad.com (saifmohammad.com)|192.185.17.122|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2474686 (2.4M) [application/zip]
Saving to: ‘WikiArt-Emotions.zip’

WikiArt-Emotions.zi 100%[===================>]   2.36M  3.35MB/s    in 0.7s    

2021-04-11 11:36:48 (3.35 MB/s) - ‘WikiArt-Emotions.zip’ saved [2474686/2474686]

Archive:  WikiArt-Emotions.zip
   creating: WikiArt-Emotions/
  inflating: WikiArt-Emotions/WikiArt-Emotions-All.tsv  
   creating: __MACOSX/
   creating: __MACOSX/WikiArt-Emotions/
  inflating: __MACOSX/WikiArt-Emotions/._WikiArt-Emotions-All.tsv  
  inflating: WikiArt-Emotions/WikiArt-annotations.csv  
  inflating: __MACOSX/WikiArt-Emotions/._WikiArt-annotations.csv  
  inflating: WikiArt-Emotions/WikiArt-info.tsv  
  inflating: __MACOSX/WikiArt-Emotions/._WikiArt-info.tsv  
  infla

### Outliers removal

In [ ]:
import pickle
with open('wikiArt/selection.pkl','rb') as f:
  selection=pickle.load(f)

In [ ]:
for image in os.listdir("wikiArt/dataset/happiness_optimism"):
  if image not in selection:
    !rm "wikiArt/dataset/happiness_optimism/{image}"

### Data augmentation

In [ ]:
for group in os.listdir("wikiArt/dataset"):
  print("{}: {}".format(group,len(os.listdir(os.path.join("wikiArt/dataset",group)))))

fear_shame: 214
anger_disgust: 67
happiness_optimism: 688
sadness_pessimism: 258


In [ ]:
data_augmentation("wikiArt/dataset",["anger_disgust","fear_shame","sadness_pessimism"],len(os.listdir("wikiArt/dataset/happiness_optimism")))

In [ ]:
for group in os.listdir("wikiArt/dataset"):
  print("{}: {}".format(group,len(os.listdir(os.path.join("wikiArt/dataset",group)))))

fear_shame: 690
anger_disgust: 487
happiness_optimism: 688
sadness_pessimism: 690


### Pre-process

In [ ]:
wikiArt_dict=preprocess_wikiart("wikiArt/dataset")
np.save("wikiArt_dict",wikiArt_dict)

### Dataloader

In [ ]:
wikiArt_dict=np.load("wikiArt_dict.npy",allow_pickle=True).item()

In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5), inplace=True)
])

BATCH_SIZE=8
wikiArt_dataloaders = {
    "anger_disgust": torch.utils.data.DataLoader(WikiArtDataLoader(wikiArt_dict["anger_disgust"],transform=transform),batch_size=BATCH_SIZE,shuffle=False,num_workers=0,drop_last=True),
    "sadness_pessimism": torch.utils.data.DataLoader(WikiArtDataLoader(wikiArt_dict["sadness_pessimism"],transform=transform),batch_size=BATCH_SIZE,shuffle=False,num_workers=0,drop_last=True),
    "fear_shame": torch.utils.data.DataLoader(WikiArtDataLoader(wikiArt_dict["fear_shame"],transform=transform),batch_size=BATCH_SIZE,shuffle=False,num_workers=0,drop_last=True),
    "happiness_optimism": torch.utils.data.DataLoader(WikiArtDataLoader(wikiArt_dict["happiness_optimism"],transform=transform),batch_size=BATCH_SIZE,shuffle=False,num_workers=0,drop_last=True)
}

## SEED-IV

In [ ]:
from seed_iv.seedIV_utils import *

### Import

We assume at this point that the user has already got the access to the SEED-IV dataset, and imported a .zip file with the following structure:

- **seed-iv.zip**
 - 1.zip
 - 2.zip
 - 3.zip
 - Channel Order.xlsx

In [ ]:
!unzip seed-iv.zip -d seed_iv && rm seed-iv.zip
!unzip seed_iv/1.zip -d seed_iv/dataset && rm seed_iv/1.zip
!unzip seed_iv/2.zip -d seed_iv/dataset && rm seed_iv/2.zip
!unzip seed_iv/3.zip -d seed_iv/dataset && rm seed_iv/3.zip

Archive:  seed_iv/1.zip
  inflating: seed_iv/dataset/1/1_20160518.mat  
  inflating: seed_iv/dataset/1/10_20151014.mat  
  inflating: seed_iv/dataset/1/11_20150916.mat  
  inflating: seed_iv/dataset/1/12_20150725.mat  
  inflating: seed_iv/dataset/1/13_20151115.mat  
  inflating: seed_iv/dataset/1/14_20151205.mat  
  inflating: seed_iv/dataset/1/15_20150508.mat  
  inflating: seed_iv/dataset/1/2_20150915.mat  
  inflating: seed_iv/dataset/1/3_20150919.mat  
  inflating: seed_iv/dataset/1/4_20151111.mat  
  inflating: seed_iv/dataset/1/5_20160406.mat  
  inflating: seed_iv/dataset/1/6_20150507.mat  
  inflating: seed_iv/dataset/1/7_20150715.mat  
  inflating: seed_iv/dataset/1/8_20151103.mat  
  inflating: seed_iv/dataset/1/9_20151028.mat  
Archive:  seed_iv/2.zip
  inflating: seed_iv/dataset/2/1_20161125.mat  
  inflating: seed_iv/dataset/2/10_20151021.mat  
  inflating: seed_iv/dataset/2/11_20150921.mat  
  inflating: seed_iv/dataset/2/12_20150804.mat  
  inflating: seed_iv/dataset/2/

### Pre-process

In [ ]:
labels = emotionDL(eps=0.1)
eeg_dict=load_data("seed_iv/dataset",labels)
np.save("eeg_dict",eeg_dict)

In [ ]:
for p in eeg_dict.keys():
  print("\nPATIENT {}".format(p))
  print("neutral:\ttrain: {:d}\tval: {:d}".format(len(eeg_dict[p]["neutral"]["train"]),len(eeg_dict[p]["neutral"]["val"])))
  print("sadness:\ttrain: {:d}\tval: {:d}".format(len(eeg_dict[p]["sadness"]["train"]),len(eeg_dict[p]["sadness"]["val"])))
  print("fear:\t\ttrain: {:d}\tval: {:d}".format(len(eeg_dict[p]["fear"]["train"]),len(eeg_dict[p]["fear"]["val"])))
  print("happiness:\ttrain: {:d}\tval: {:d}".format(len(eeg_dict[p]["happiness"]["train"]),len(eeg_dict[p]["happiness"]["val"])))


PATIENT 11
neutral:	train: 448	val: 230
sadness:	train: 432	val: 251
fear:		train: 419	val: 196
happiness:	train: 388	val: 141

PATIENT 8
neutral:	train: 448	val: 230
sadness:	train: 432	val: 251
fear:		train: 419	val: 196
happiness:	train: 388	val: 141

PATIENT 10
neutral:	train: 448	val: 230
sadness:	train: 432	val: 251
fear:		train: 419	val: 196
happiness:	train: 388	val: 141

PATIENT 3
neutral:	train: 448	val: 230
sadness:	train: 432	val: 251
fear:		train: 419	val: 196
happiness:	train: 388	val: 141

PATIENT 12
neutral:	train: 448	val: 230
sadness:	train: 432	val: 251
fear:		train: 419	val: 196
happiness:	train: 388	val: 141

PATIENT 6
neutral:	train: 448	val: 230
sadness:	train: 432	val: 251
fear:		train: 419	val: 196
happiness:	train: 388	val: 141

PATIENT 13
neutral:	train: 448	val: 230
sadness:	train: 432	val: 251
fear:		train: 419	val: 196
happiness:	train: 388	val: 141

PATIENT 2
neutral:	train: 448	val: 230
sadness:	train: 432	val: 251
fear:		train: 419	val: 196
happiness:	

### Dataloader

In [ ]:
eeg_dict=np.load("eeg_dict.npy",allow_pickle=True).item()

In [ ]:
adjacency_matrix=get_adjacency_matrix("seed_iv/Channel Order.xlsx","seed_iv/Channel Location.txt")
adjacency_matrix=torch.tensor(adjacency_matrix, dtype=torch.float32)

/content/Emotions-fromEEG-toArt/seed_iv/seedIV_utils.py:103: RuntimeWarning: divide by zero encountered in true_divide
  adjacency_matrix = np.minimum(np.ones([62,62]), delta/(distances_matrix**2))


In [ ]:
BATCH_SIZE=8
eeg_dataloaders = {}
for p in eeg_dict.keys():
  eeg_dataloaders[p]={
    "neutral": {
      "train": torch_geometric.data.DataLoader(eeg_dict[p]["neutral"]["train"],batch_size=BATCH_SIZE,shuffle=False,drop_last=True),
      "val": torch_geometric.data.DataLoader(eeg_dict[p]["neutral"]["val"],batch_size=BATCH_SIZE,shuffle=False,drop_last=True)
    },
    "sadness": {
      "train": torch_geometric.data.DataLoader(eeg_dict[p]["sadness"]["train"],batch_size=BATCH_SIZE,shuffle=False,drop_last=True),
      "val": torch_geometric.data.DataLoader(eeg_dict[p]["sadness"]["val"],batch_size=BATCH_SIZE,shuffle=False,drop_last=True)
    },
    "fear": {
      "train": torch_geometric.data.DataLoader(eeg_dict[p]["fear"]["train"],batch_size=BATCH_SIZE,shuffle=False,drop_last=True),
      "val": torch_geometric.data.DataLoader(eeg_dict[p]["fear"]["val"],batch_size=BATCH_SIZE,shuffle=False,drop_last=True)
    },
    "happiness": {
      "train": torch_geometric.data.DataLoader(eeg_dict[p]["happiness"]["train"],batch_size=BATCH_SIZE,shuffle=False,drop_last=True),
      "val": torch_geometric.data.DataLoader(eeg_dict[p]["happiness"]["val"],batch_size=BATCH_SIZE,shuffle=False,drop_last=True)
    }
  }

## StyleGAN2-ADA

In [ ]:
#IDENTIFY THE VERSION TO COPY-PASTE